In [ ]:
import pandas as pd
import random

In [ ]:
#a placeholder dataset. In the future this app will scrape the local, daily events of EventBrite.
df = pd.read_csv('LA.csv')
df = df.dropna(subset=['Event Name'])
#the questionaire filled out by users
friends_df = pd.read_csv('friendship_Q')
#pairing table
relationships = pd.read_csv('relationships.csv')

In [ ]:
df['Event Name'] = df['Event Name'].str.lower()
df['Event Description'] = df['Event Description'].str.lower()

df = df[df['Event Name'].notna()]

df = df[~df['Event Name'].str.contains('kids')]
df = df[~df['Event Name'].str.contains('office')]
df = df[~df['Event Name'].str.contains('teen')]
df = df[~df['Event Name'].str.contains('mtg')]
df = df[~df['Event Name'].str.contains('board')]
df = df[~df['Event Name'].str.contains('meeting')]
df = df[~df['Event Name'].str.contains('child')]
df = df[~df['Event Name'].str.contains('job')]




In [ ]:
friends_df = friends_df.fillna(0)

In [ ]:
#number to user
def n2u(number):
    return friends_df.iloc[number]

Dictionary for relating events to features

In [ ]:
event_features_dict = {'active': ['sail', 'run', 'marathon', 'workout', 'yoga', 'hike', 'gym', 'dance' 'swim', 'jog'],
                       'food': ['food', 'cuisine', 'food festival', 'delicious'],
                       'virtual': ['virtual', 'zoom'],
                       'music': ['music', 'band', 'concert'],
                       'draw' : ['paint', 'art', 'photography', 'crafts', 'beads'],
                       'movie' : ['film', 'movie', 'netflix', 'screening'],
                      'brain': ['lecture', 'talk', 'science', 'discussion'],
                      'food': ['tasting', 'drinks', 'delicious', 'restaurant', 'food festival', 'eat', 'dine', 'cooking'],
                      'comedy': ['funny', 'standup', 'stand-up', 'stand up', 'comedy'],
                      'writer': ['author', 'library', 'book', 'poetry', 'writing workshop']}
keys_events = list(event_features_dict.keys())
vals_events = list(event_features_dict.values())

In [ ]:
friends_df['What is your preferred way of spending time with friends?'].iloc[0]

'Grabbing foods or drinks together, Doing an activity or exploring, Playing sport, painting, picnicing, going out to the city, etc.'

In [ ]:
#finds out if they will like to do an activity together
def together(user1, user2, str):
    letsdoit = 0
    if (str in friends_df['What is your preferred way of spending time with friends?'].iloc[user1]):
        if (str in friends_df['What is your preferred way of spending time with friends?'].iloc[user2]):
            letsdoit = random.randint(5, 10)
    return letsdoit
            
            
            

In [ ]:
relationships.columns

Index(['Unnamed: 0', 'User1', 'User2', 'Suggested Activity'], dtype='object')

In [ ]:
#returns most compatible activity
def most_compat(u1, u2):
    user1 = n2u(u1)
    user2 = n2u(u2)
    
    active = user1['Rank these activities on how much you enjoy them? (5 is most enjoyable) [Hiking]'] + user2['Rank these activities on how much you enjoy them? (5 is most enjoyable) [Hiking]']
    
    draw = user1['Rank these activities on how much you enjoy them? (5 is most enjoyable) [Drawing]'] + user2['Rank these activities on how much you enjoy them? (5 is most enjoyable) [Drawing]']
    brain = user1['Rank these activities on how much you enjoy them? (5 is most enjoyable) [Reading nonfiction]'] + user2['Rank these activities on how much you enjoy them? (5 is most enjoyable) [Reading nonfiction]']
    food = together(u1, u2, 'Grabbing foods or drinks together')
    virtual = together(u1, u2, 'Electronically: messaging, calling, video-chatting, etc.')
    writer = together(u1, u2, 'Rank these activities on how much you enjoy them? (5 is most enjoyable) [Journaling]')
    
    movies = user1['Netflix party'] + user2['Netflix party']
    
    
    dictionary = { 'active' : active, 'draw': draw, 'brain': brain, 'food': food, 'virtual': virtual, 'writer': writer,
                 'movie': movies}
    
    maxy = max(active, draw, brain, food, virtual)
    keys = list(dictionary.keys())
    vals = list(dictionary.values())
    
    #randomly picks from top 3
    
    return keys[vals.index(maxy)]






In [ ]:
def reccomend_something(u1, u2):
    
    user1 = n2u(u1)
    user2 = n2u(u2)
    
    compatible_activity = most_compat(u1, u2)
    
    
    lw = event_features_dict.get(most_compat(u1, u2))
    new_df = df.copy()
    #new_df = if_in_list(lw, new_df.loc[new_df['Event Name']])
    mask = new_df['Event Name'].apply(lambda x: any(item for item in lw if item in x))
    df1 = df[mask]
    
    ans = df1.sample()[['Event Name']]
    return ans.iloc[0]

In [ ]:
reccomend_something(relationships['User1'].iloc[15], relationships['User2'].iloc[20])

Event Name    all-ages hatha yoga
Name: 24846, dtype: object

In [ ]:
#relationships.apply(lambda x: reccomend_something(relationships['User1'], relationships['User2']), axis=1)
relationships['Suggested Activity'] = relationships.apply(lambda x: reccomend_something(x['User1'], x['User2']), axis=1)


In [ ]:
relationships.head(30)

,Unnamed: 0,User1,User2,Suggested Activity
0,0,0,1,"ballet, hip hop, piano, gymnastics, cheerleadi..."
1,1,0,2,"ballet, hip hop, piano, gymnastics, cheerleadi..."
2,2,0,3,greater valley glen nc_planning & land use com...
3,3,0,4,"ballet, hip hop, piano, gymnastics, cheerleadi..."
4,4,0,5,dahn yoga classes
5,5,0,6,"ballet, hip hop, piano, gymnastics, cheerleadi..."
6,6,0,7,"ballet, hip hop, piano, gymnastics, cheerleadi..."
7,7,0,8,"recreation and parks: august 10, 2016 facility..."
8,8,0,9,greater cypress park outreach committee agenda...
9,9,0,10,pajama storytime double feature: tail waggin' ...
